In [1]:
import csv
import numpy as np
from shapely.geometry import Polygon, Point

#### We focus on Subduction zones and extract pertinent subcatalogues with following criterea:

- Magnitude Mw >=3.0
- Spatial bounds of subduction zones
- Depths < 500 km

In [2]:
def is_inZone(x,y, szone, subduction_bounds):
    xypoint = Point(x, y)
    if xypoint.within(subduction_bounds[szone]):
        return True
    return False

def extract_subcatalogue(input_catalogue, catalogue_format, output_file, 
                         Mmin=3.0, Hmax = 500, subduction_bounds= None):
    
    if subduction_bounds is None:
        hik_finterp = np.load('../../1SubductionModel/finterp/hik_finterp.npy', allow_pickle=True)[()]
        hik_bounds = hik_finterp['xbounds']
        puy_finterp = np.load('../../1SubductionModel/finterp/puy_finterp.npy', allow_pickle=True)[()]
        puy_bounds = puy_finterp['xbounds']
        subduction_bounds = {'hik': hik_bounds, 'puy': puy_bounds}
    #
    fout = open(output_file, 'w')
   
    fout.write("year, month, day, hour, minute, second, lon, lat, depth, mag")
    #fout.write("date,time,mag,magtype,decimalyear,dayofyear,X,Y,Z,lon,lat,depth\n")
    
    with open(input_catalogue, "r") as f:
        datareader=csv.reader(f)
        header = next(datareader)
        nrow = 0
        for row in datareader: 
            if catalogue_format=='rollins':
                lon = float(row[6])
                lat = float(row[7])
                dep = float(row[8])
                mag = float(row[9])
            elif data_format =='rastin':
                mag = float(row[2])
                lon = float(row[6])
                lat = float(row[7])
                dep = float(row[8])
            else:
                raise Exception('Error: please, the format is either rollins or rastin')
            
            if mag<Mmin:
                continue
            if dep>Hmax:
                continue
            
            # check subduction 
            in_zone =False
            if is_inZone(lon,lat, 'hik', subduction_bounds):
                in_zone = True
            elif is_inZone(lon,lat, 'puy', subduction_bounds):
                in_zone = True
            
            if not in_zone:
                continue
                
            if catalogue_format=='rollins':
                fout.write('\n%d, %d, %d, %d, %d, %.3f, ' \
                       %(int(row[0]), int(row[1]), int(row[2]), \
                         int(row[3]), int(row[4]), float(row[5])))
                fout.write('%.4f, %.4f, %.2f, %.3f' %(lon, lat, dep, mag))
            else:            
                tdate = row[0].split('-')
                ttime = row[1].split(':')
                fout.write('\n%d, %d, %d, %d, %d, %.3f, ' \
                       %(int(tdate[2]), int(get_dmonth(tdate[1])), int(tdate[0]), \
                         int(ttime[0]), int(ttime[1]), float(ttime[2])))
                fout.write('%.4f, %.4f, %.2f, %.3f' %(lon, lat, dep, mag))
    fout.close()
    print('Extracted catalogue:', output_file)

def get_dmonth(mstr):
    if mstr.lower() =='jan':
        return 1
    if mstr.lower() =='feb':
        return 2
    if mstr.lower() =='mar':
        return 3
    if mstr.lower() =='apr':
        return 4
    if mstr.lower() =='may':
        return 5
    if mstr.lower() =='jun':
        return 6
    if mstr.lower() =='jul':
        return 7
    if mstr.lower() =='aug':
        return 8
    if mstr.lower() =='sep':
        return 9
    if mstr.lower() =='oct':
        return 10
    if mstr.lower() =='nov':
        return 11
    if mstr.lower() =='dec':
        return 12
    raise Exception('mstr is not a Month:' + mstr)
    

In [3]:
subduction_bounds = np.load('../subduction_model/subduction_bounds.npy', allow_pickle=True)[()]

data_format = "rollins"
input_file = "catalogs/NZeqcat_Rollins13042020.csv"
output_file = "subduction_catalogs/NZeqcat_Rollins13042020-subd.csv"

extract_subcatalogue(input_file, data_format, output_file)

Extracted catalogue: subduction_catalogs/NZeqcat_Rollins13042020-subd.csv


In [4]:
data_format = "rastin"
input_file = "catalogs/Grue_NZeqcat_Rollins13042020SepiDeep.csv"
output_file = "subduction_catalogs/Grue_NZeqcat_Rollins13042020SepiDeep-subd.csv"
extract_subcatalogue(input_file, data_format, output_file)

Extracted catalogue: subduction_catalogs/Grue_NZeqcat_Rollins13042020SepiDeep-subd.csv


In [6]:
data_format = "rastin"
input_file = "catalogs/Grue_NZeqcat_Rollins13042020SepiShallow.csv"
output_file = "subduction_catalogs/Grue_NZeqcat_Rollins13042020SepiShallow_subd.csv"
extract_subcatalogue(input_file, data_format, output_file)

Extracted catalogue: subduction_catalogs/Grue_NZeqcat_Rollins13042020SepiShallow_subd.csv
